In [2]:
import copy
import glob
import gc
import os
import sys
import warnings
import os
import sys
import math
import time
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from data import TechDataset, CVSampler
from model import Encoder_SEQ, Decoder_SEQ, SEQ2SEQ
from train_utils import run_epoch, EarlyStopping, perf_eval

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [110]:
TOKEN_SOS = '<SOS>'
TOKEN_EOS = '<EOS>'
TOKEN_PAD = '<PAD>'
tokens = [TOKEN_SOS, TOKEN_EOS, TOKEN_PAD]
regex = re.compile("[0-9a-zA-Z]+")

In [4]:
data_root = "/home2/glee/Tech_Gen/data/"
rawdata = pd.read_csv(os.path.join(data_root, "collection_final.csv"))
rawdata_dropna = rawdata.dropna(axis=0, subset=['main ipc', 'sub ipc'])[['number','main ipc', 'sub ipc']]
cols_year = ['<1976']+list(np.arange(1976,2018).astype(str))
n_TC = 3

In [5]:
ipc, num = np.unique(rawdata_dropna['main ipc'].apply(lambda x: x.split(' ')[0]), return_counts=True)
ipc_vocab_size = pd.concat([pd.Series(ipc[np.argsort(num)[::-1]]), pd.Series(num[np.argsort(num)[::-1]])], axis=1)

In [7]:
ipc_vocab_size.iloc[:20]

,0,1
0,A61K,44658
1,C07K,7686
2,A01N,6265
3,C12N,2174
4,A61L,2008
5,A61F,1841
6,A23L,1075
7,G01N,820
8,A23K,767
9,C07D,692


In [111]:
target_ipc = "A61K"

In [113]:
main_ipcs = [x for x in pd.unique(rawdata_dropna['main ipc']) if target_ipc in x]
rawdata_ipc = rawdata_dropna.loc[rawdata_dropna['main ipc'].isin(main_ipcs)]
data = rawdata_ipc[['number']].copy(deep=True)
data['main_ipc'] = rawdata_ipc['main ipc'].apply(lambda x: regex.findall(x)[0])
data['sub_ipc'] = rawdata_ipc['sub ipc'].apply(lambda x: [regex.findall(xx)[0] for xx in x.split(';')])

rawdata_tc = rawdata.loc[rawdata_ipc.index][['year']+cols_year]
data['TC'+str(n_TC)] = rawdata_tc.apply(lambda x: x[np.arange(x['year']+1 if x['year']<2017 else 2017, x['year']+n_TC+1 if x['year']+n_TC<2018 else 2018).astype(str)].sum(), axis=1)

data = data.set_index('number')
# main_ipcs = [regex.findall(x)[0] for x in main_ipcs]
main_ipcs = [target_ipc]
sub_ipcs = list(np.unique(np.concatenate(list(data['sub_ipc'].values))))
all_ipcs = list(np.union1d(main_ipcs, sub_ipcs))
seq_len = data['sub_ipc'].apply(lambda x: len(x)).max() + 3

vocab_w2i = {all_ipcs[i]: i for i in range(len(all_ipcs))}
vocab_w2i.update({tokens[i]: len(all_ipcs)+i for i in range(len(tokens))})
vocab_i2w = {i: all_ipcs[i] for i in range(len(all_ipcs))}
vocab_i2w.update({len(all_ipcs)+i: tokens[i] for i in range(len(tokens))})
vocab_size = len(vocab_w2i)

In [36]:
X_df = pd.DataFrame(index=data.index)
X_df['main'] = data['main_ipc'].apply(lambda x: vocab_w2i[x])
X_df['sub'] = data['sub_ipc'].apply(lambda x: [vocab_w2i[xx] for xx in x])
main_sub_combined = X_df.apply(lambda x: [x['main']]+x['sub'], axis=1)
X_df['seq'] = main_sub_combined.apply(lambda x: np.concatenate([[vocab_w2i['<SOS>']]+x+[vocab_w2i['<EOS>']], np.zeros(seq_len-(len(x)+2))+vocab_w2i['<PAD>']]).astype(int))

# xaxis = np.concatenate([np.tile([i], X_df['sub'].apply(lambda x: len(x)).values[i]) for i in range(len(X_df))])
# X = np.zeros((len(self.data), len(self.all_ipcs))) # (#samples, #ipcs)
# X[tuple(np.arange(len(X_df))), tuple(X_df['main'].values)] += 10
# X[tuple(xaxis), tuple(np.concatenate(X_df['sub'].values))] += 1

# X = np.zeros((len(data), seq_len, len(all_ipcs)+len(tokens)))
# X[tuple(np.sort(np.tile(np.arange(len(X_df)), seq_len))), tuple(np.tile(np.arange(seq_len), len(X_df))), tuple(np.concatenate(X_df['seq'].values))] += 1

In [520]:
batch_size = 32
learning_rate = 0.001

In [538]:
tech_dataset.data

,main_ipc,sub_ipc,TC3
number,,,
9040668,A61K39/395,"[C07K19/00, C07K14/765, C12P21/08, C07K16/46, ...",0
9040626,A61K8/02,"[A61K9/70, B82Y5/00]",0
9040579,A61K31/216,[A61K31/122],0
9040103,A61K36/00,"[A61K36/53, A61K31/045]",0
9040102,A61K36/539,"[A61K36/48, A61K36/00, A61K36/31]",0
...,...,...,...
3932610,A61K8/72,"[A61K8/60, A61K8/81, A61K8/46, A61K8/92, A61K8...",2
3932607,A61K33/16,"[A61K8/43, A61K8/44, A61K8/30, A61Q11/00]",0
3932603,A61K8/64,"[A61K8/37, A61K8/46, A61K8/44, A61K8/96, A61K8...",0


In [312]:
print("TRANSFORM one-by-one")
tstart = time.time()
tech_dataset = TechDataset(device=device, data_dir=data_root, do_transform=True, params={'target_ipc': 'A61K'})
data_loader = DataLoader(tech_dataset, batch_size=batch_size)
tend = time.time()
print(f"{tend-tstart} sec Elapsed")

print("TRANSFORM as a whole")
tstart = time.time()
tech_dataset = TechDataset(device=device, data_dir=data_root, do_transform=False, params={'target_ipc': 'A61K'})
data_loader = DataLoader(tech_dataset, batch_size=batch_size)
tend = time.time()
print(f"{tend-tstart} sec Elapsed")

TRANSFORM one-by-one
29.3840069770813 sec Elapsed
TRANSFORM as a whole
28.631096363067627 sec Elapsed


In [323]:
tstart = time.time()
xx, yy = next(iter(data_loader))
tend = time.time()
print(f"{tend-tstart} sec Elapsed")

0.0036962032318115234 sec Elapsed


In [339]:
enc = Encoder_SEQ(embedding_dim=128, vocab_size=vocab_size, hidden_dim=32, n_layers=1, device=device, padding_idx=tech_dataset.vocab_w2i['<PAD>'])
enc = enc.to(dtype=torch.float)

In [340]:
enc(xx)

tensor([[[-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351],
         [-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351],
         [-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351],
         ...,
         [-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351],
         [-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351],
         [-0.0181, -0.0901,  0.1597,  ...,  0.1031, -0.0934, -0.1351]]],
       grad_fn=<StackBackward>)

In [386]:
dec = Decoder_SEQ(embedding_dim=128, vocab_size=vocab_size, hidden_dim=32, n_layers=1, device=device)

In [387]:
next_input = torch.from_numpy(np.tile(vocab_w2i[TOKEN_SOS], 32)).unsqueeze(1)

In [371]:
dec.initHidden(len(next_input)).shape

torch.Size([1, 32, 32])

In [388]:
o,h = dec(next_input, hidden=None)

In [389]:
o.shape

torch.Size([32, 6032])

In [466]:
seq2seq = SEQ2SEQ(device=device, dataset=tech_dataset, enc=enc, dec=dec, max_len=tech_dataset.seq_len)

In [467]:
outputs, zz = seq2seq(xx)

In [481]:
outputs.shape

torch.Size([168, 32, 6032])

In [480]:
np.unique([tech_dataset.vocab_i2w[i] for i in outputs[:,0,:].argmax(1).detach().numpy()])

array(['A01C1/06', 'A23K1/14', 'A61B5/055', 'A61K31/06', 'A61K31/133',
       'A61K31/53', 'A61K31/713', 'A61K35/64', 'A61K36/63', 'A61K39/085',
       'A61K47/40', 'A61K8/44', 'A61M5/20', 'A61P15/00', 'B01J19/10',
       'B01J39/14', 'B05D7/00', 'B32B5/24', 'C01B15/037', 'C01B21/24',
       'C03C3/15', 'C07B39/00', 'C07B59/00', 'C07C229/24', 'C07C239/00',
       'C07C25/00', 'C07C275/10', 'C07C275/40', 'C07C43/315', 'C07C59/72',
       'C07C69/28', 'C07D207/27', 'C07D241/24', 'C07D253/00',
       'C07D261/20', 'C07D277/34', 'C07D295/32', 'C07D305/08',
       'C07D487/18', 'C07F9/44', 'C07K1/113', 'C07K14/555', 'C08F2/50',
       'C08F224/00', 'C08G65/332', 'C08G77/20', 'C08G77/38', 'C08G77/458',
       'C08J3/03', 'C09C1/62', 'C09J133/02', 'C09K15/24', 'C12N1/20',
       'C12N5/0775', 'C12P17/12', 'C13K5/00', 'C40B30/04', 'G01N21/80',
       'G01N33/573'], dtype='<U10')

In [485]:
loss_fn = torch.nn.CrossEntropyLoss()

In [486]:
loss_fn

CrossEntropyLoss()

In [510]:
loss_fn(outputs.transpose(0,1).transpose(1,2), xx)

tensor(8.7048, grad_fn=<NllLoss2DBackward>)

In [487]:
xx

torch.Size([32, 168])

In [516]:
trues = xx.clone()

In [521]:
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=learning_rate)

In [522]:
batch_losses = []

outputs, z = seq2seq(xx) # outputs shape: (seq_len, batch_size, vocab_size)
preds = outputs.transpose(0,1).transpose(1,2) # preds shape: (batch_size, vocab_size, seq_len), regard seq_len as additional dimension
trues = xx.clone()
loss = loss_fn(preds, trues)
batch_losses.append(loss.item())

optimizer.zero_grad()
loss.backward()
optimizer.step()

# if batch % 10 == 0 or batch == len(dataloader)-1:
#     loss, current = loss.item(), batch*len(X)
#     if batch == len(dataloader)-1:
#         current = size
#     print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]", end='\r', flush=True)

In [526]:
max_epochs = 5

In [537]:
early_stopping = EarlyStopping(patience=10, verbose=True, path="../models/ES_checkpoint.ckpt")
for ep in range(max_epochs):
    print(f"Epoch {ep+1}\n"+str("-"*30))
    train_loss = run_epoch(data_loader, seq2seq, loss_fn, mode='train', optimizer=optimizer)

Epoch 1
------------------------------


KeyboardInterrupt: 

In [546]:
sampler = CVSampler(tech_dataset, n_folds=1, test_ratio=0.3)

In [552]:
cv_idx = sampler.get_idx_dict()

In [553]:
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

#Samples
Train: 25008, Validation: 6252, Test: 13398


In [545]:
import importlib
import data
importlib.reload(data)
from data import TechDataset, CVSampler

import model
importlib.reload(model)
from model import Encoder_SEQ, Decoder_SEQ, SEQ2SEQ

import train_utils
importlib.reload(train_utils)
from train_utils import run_epoch, EarlyStopping, perf_eval